<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>

<p>Import the required libraries.</p>

In [81]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

<p>From Wikipedia, get an html page that contains a list of postal codes for Toronto.</p>

In [82]:
wiki_link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
raw_wiki_page = requests.get(wiki_link)
page_text = raw_wiki_page.text

<p>Parse the page using BeautifulSoup.</p>

In [83]:
soup = BeautifulSoup(page_text, "html.parser")

<p>Find the table body, then select all the rows.</p>

In [84]:
codes_table = soup.find("tbody")
table_rows = codes_table.find_all('tr')

<p>Iterate through the rows. For each cell in each row, grab the text, append it to a new list (rows) then append the list to the data list.</p>

In [85]:
data = [[], [], []]
columns = ["PostalCode", "Borough", "Neighborhood"]
for row in table_rows:
    data_cells = row.find_all('td')
    rows = []
    for cell in data_cells:
         rows.append(cell.get_text(strip=True))
    data.append(rows)

<p>Create a dataframe from the data list. Remove rows where Borough is Not Assigned. Remove rows with values of None. Where Neighborhood is Not Assigned, replace with the value from the Borough column.</p>

In [120]:
df = pd.DataFrame(data=data, columns=columns)
df = df[df.Borough != "Not assigned"]
df = df.dropna()
df = df.reset_index(drop=True)
df['Neighborhood'] = np.where(df['Neighborhood'] == "Not assigned", df['Borough'], df['Neighborhood'])


<p>Define a method which joins the Neighborhoods for common Boroughs into a String. Group the Boroughs and run the create_list method.</p>

In [124]:
def create_list(x):
    return ', '.join(x)

df = df.groupby('Borough', as_index=False).agg({'PostalCode': 'first', 'Neighborhood': create_list})
df

,Borough,PostalCode,Neighborhood
0,Central Toronto,M4N,"Lawrence Park, Roselawn, Davisville North, For..."
1,Downtown Toronto,M5A,"Harbourfront, Regent Park, Ryerson, Garden Dis..."
2,East Toronto,M4E,"The Beaches, The Danforth West, Riverdale, The..."
3,East York,M4B,"Woodbine Gardens, Parkview Hill, Woodbine Heig..."
4,Etobicoke,M9A,"Islington Avenue, Cloverdale, Islington, Marti..."
5,Mississauga,M7R,Canada Post Gateway Processing Centre
6,North York,M3A,"Parkwoods, Victoria Village, Lawrence Heights,..."
7,Queen's Park,M7A,Queen's Park
8,Scarborough,M1B,"Rouge, Malvern, Highland Creek, Rouge Hill, Po..."
9,West Toronto,M6H,"Dovercourt Village, Dufferin, Little Portugal,..."


In [125]:
df.shape

(11, 3)